<a href="https://colab.research.google.com/github/MiguelSteph/email-labeling-with-ai/blob/main/email_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install missing packages

In [1]:
!pip install -U -q 'google-api-python-client==1.7.2' 'google-auth-httplib2==0.0.3' 'google-auth-oauthlib==0.4.1' 'google-genai>=1.51.0'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.6/426.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.3/233.3 kB 6.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.45.0 which is incompatible.
pydrive2 1.21.3 requires google-api-python-client>=1.12.5, but you have google-api-python-client 1.7.2 which is incompatible.
earthengine-api 1.5.24 requires google-api-python-client>=1.12.1, but you have google-api-python-client 1.7.2 which is incompatible.
firebase-admin 6.9.0 requires google-api-python-client>=1.7.8, but you have google-api-python-client 1.7.2 which is incompatible.
google-adk 1.2

In [2]:
import os
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import pandas as pd
import re
import time
import math
import numpy as np
from urllib.parse import unquote
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from hdbscan import HDBSCAN

import base64
from bs4 import BeautifulSoup

from google import genai
from google.genai import types
from google.colab import userdata

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Change the credential path to point to your credentials file.
CREDENTIALS_FILE_PATH = "/content/drive/MyDrive/Projects/email_labeling/credentials.json"
RAW_DATASET_PATH = "/content/drive/MyDrive/Projects/email_labeling/raw_dataset.pkl"
DATASET_WITH_EMBEDDINGS_PATH = "/content/drive/MyDrive/Projects/email_labeling/dataset_with_embeddings.pkl"


/usr/local/lib/python3.12/dist-packages/hdbscan/robust_single_linkage_.py:175: SyntaxWarning: invalid escape sequence '\{'
  $max \{ core_k(a), core_k(b), 1/\alpha d(a,b) \}$.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Get user access

In [3]:
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

def get_user_access():
  """Get the user's access and refresh tokens
  and save them in a token.json file.
  """
  creds = None
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE_PATH,
                                                       SCOPES)
      # creds = flow.run_local_server(port=0)
      creds = flow.run_console()
    with open("token.json", "w") as token:
      token.write(creds.to_json())
    return creds


# Get credentials
creds = get_user_access()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=201451649192-58mbdfjc2kdgj591d5qj090jditt0rbg.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=KnEQTzENl9dgGODq453DiX0FDpKzPR&prompt=consent&access_type=offline
Enter the authorization code: 4/1ATX87lPo-fj9h9gQvy0aDJNqVAAR6099NAMZguOu13kUv3mizhzy1IuOKJY


# Prepare raw dataset

Follow the steps here https://developers.google.com/workspace/gmail/api/quickstart/python to configure and download the credentials.

## Fetch and save emails from Gmails

In [ ]:
def decode_base64_str(encoded_str):
  return base64.urlsafe_b64decode(encoded_str + '=' * (4 - len(encoded_str) % 4)).decode('utf-8')


def get_email_content(msgs):
  for m in msgs:
    mimeType = m['mimeType']
    if (mimeType == 'text/plain'):
      return decode_base64_str(m['body']['data'])

    if (mimeType == 'text/html'):
      html_text = decode_base64_str(m['body']['data'])
      return BeautifulSoup(html_text, 'html.parser').get_text()

    if 'parts' in m:
      return get_email_content(m['parts'])


def get_emails(credentials, num_emails):
  max_emails_per_fetch = 500
  try:
    email_fetched = 0
    page_token = ""
    ids = []
    thread_ids = []
    emails_content = []
    while email_fetched < num_emails:
      fetch_count = min(num_emails - email_fetched, max_emails_per_fetch)
      service = build("gmail", "v1", credentials=credentials)
      results = service.users().messages().list(userId="me",
                                                labelIds=["INBOX"],
                                                includeSpamTrash=False,
                                                # maxResults=fetch_count,
                                                pageToken=page_token).execute()
      messages = results.get("messages", [])
      page_token = results.get("nextPageToken", "")

      for message in messages:
        msg =  service.users().messages().get(userId="me", id=message["id"], format="full").execute()
        content = get_email_content([msg['payload']])
        content = content.replace('\r', '')
        content = content.replace('\n', '')
        ids.append(message["id"])
        thread_ids.append(message["threadId"])
        emails_content.append(content)

      email_fetched += len(messages)
    return ids, thread_ids, emails_content
  except HttpError as error:
    print(f"An error occurred: {error}")

In [ ]:
"""
  - Fetch emails
  - Put them in a dataframe
  - Save the dataframe as a pickle file
"""
ids, thread_ids, emails_content = get_emails(creds, 1000)
df = pd.DataFrame({
      "ids": ids,
      "thread_ids": thread_ids,
      "body": emails_content
    })

# Save raw dataset
df.to_pickle(RAW_DATASET_PATH)

## Load and clean raw dataset

In [ ]:
MAX_CONTENT_CHARACTERS = 10_000

# Load raw dataset
df = pd.read_pickle(RAW_DATASET_PATH)

# Remove all the links from the emails content
df['body'] = df['body'].map(lambda email: unquote(email))
df['body'] = df['body'].map(lambda email: re.sub(r'http\S+', ' ', email))
# Remove HTML tag
df['body'] = df['body'].map(lambda email: BeautifulSoup(email, 'html.parser').get_text())
# Remove consecutive punctuations and replace them with only the first one
df['body'] = df['body'].map(lambda email: re.sub(r'(\W)(?=\1)', ' ', email))

# Remove HTML special characters
def remove_hmtl_special_characters(email):
  email = email.replace('&#8192;', '')
  email = email.replace('&#8193;', '')
  email = email.replace('&#8194;', '')
  email = email.replace('&#8195;', '')
  email = email.replace('&#8196;', '')
  email = email.replace('&#8197;', '')
  email = email.replace('&#8198;', '')
  email = email.replace('&#8199;', '')
  email = email.replace('&#8200;', '')
  email = email.replace('&#8201;', '')
  email = email.replace('&#8202;', '')
  email = email.replace('&#8203;', '')
  email = email.replace('&#8204;', '')
  email = email.replace('&#8205;', '')
  email = email.replace('&#8206;', '')
  email = email.replace('&#8207;', '')
  email = email.replace('&#8239;', '')
  email = email.replace('&zwnj;', '')
  email = email.replace('&#847;', '')
  return email

df['body'] = df['body'].map(remove_hmtl_special_characters)

# Replace multiple spaces with a single one
df['body'] = df['body'].map(lambda email: ' '.join(email.split()))

# Removes rows with empty or very small email body
df['body'] = df['body'].map(lambda email: email if len(email) > 40 else '')
empty_content_indexes = df[df['body'] == ''].index.tolist()
df = df.drop(labels=empty_content_indexes)
df = df.reset_index(drop=True)

# Clip the email body to maximum 10_000 characters
df['body'] = df['body'].map(lambda email: email if len(email) < MAX_CONTENT_CHARACTERS else email[:MAX_CONTENT_CHARACTERS])

In [ ]:
# # Select the first 5 emails for testing purposes
# df = df[:10]
# print(df)

# Embed raw emails

In [ ]:
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
# GEMINI_API_KEY = userdata.get('GEMINI_PAID_TIER_API_KEY')
MODEL_NAME = "gemini-embedding-001"
client = genai.Client(api_key=GEMINI_API_KEY)

def count_tokens(contents):
  result = client.models.count_tokens(
      model=MODEL_NAME,
      contents=contents,
  )
  return result.total_tokens

def get_embedding(contents):
  result = client.models.embed_content(
      model=MODEL_NAME,
      contents=contents,
      config=types.EmbedContentConfig(task_type="CLUSTERING")
  )
  embeddings = [embedding.values for embedding in result.embeddings]
  return np.array(embeddings).mean(axis=0)

## Counting tokens

In [ ]:
MAX_CHUNK_CHARACTERS = 2048

def maybe_split_content(content):
  content_len = len(content)
  if content_len <= MAX_CHUNK_CHARACTERS:
    return [content]
  words_chunk = content.split()
  chunks = []
  current_chunks = []
  current_len = 0
  for word in words_chunk:
    word_len = len(word)
    if current_len + word_len <= MAX_CHUNK_CHARACTERS:
      current_chunks.append(word)
      current_len += word_len
    else:
      chunk = ' '.join(current_chunks)
      chunks.append(chunk)
      current_chunks = [word]
      current_len = word_len
  if len(current_chunks) > 0:
    chunk = ' '.join(current_chunks)
    chunks.append(chunk)
  return chunks



tokens_counts = []
total_tokens = 0
for email in df['body']:
  contents = maybe_split_content(email)
  request_token_count = count_tokens(contents)
  total_tokens += request_token_count
  tokens_counts.append(request_token_count)

print(f"Max token per request is {max(tokens_counts)}")
print(f"Total tokens count is {total_tokens}")

Max token per request is 4612
Total tokens count is 651929


## Get embeddings

In [ ]:
embeddings = []
error_indexes = []

for idx, email in enumerate(df['body']):
  try:
    contents = maybe_split_content(email)
    embedding = get_embedding(contents)
    embeddings.append(embedding)
  except:
    error_indexes.append(idx)

  # Pause for 1 minute after 120 requests
  if idx > 0 and idx%120 == 0:
    time.sleep(60)

# Drop the emails for which we fail to get the embeddings
df = df.drop(labels=error_indexes)
df = df.reset_index(drop=True)

# Add the embeddings in the dataframe
df['embedding'] = embeddings

# Save dataset with embeddings
df.to_pickle(DATASET_WITH_EMBEDDINGS_PATH)

if len(error_indexes) > 0:
  print(f"The following rows have been removed: {error_indexes}")
else:
  print(f"No row have been removed.")

No row have been removed.


# Cluster Emails

In [4]:
REDUCED_DIM = 10
SEED = 42

# Load emails with embeddings
df = pd.read_pickle(DATASET_WITH_EMBEDDINGS_PATH)

# Reduce the dimensionality of the emails
umap_model = UMAP(n_components=REDUCED_DIM, min_dist=0.0,
                  metric='cosine', random_state=SEED)
reduced_embeddings = umap_model.fit_transform(df['embedding'].tolist())

# Cluster the emails
hdbscan_model = HDBSCAN(min_cluster_size=40,
                        metric='euclidean',
                        cluster_selection_method='eom').fit(reduced_embeddings)
clusters = hdbscan_model.labels_
clusters = [int(cluster_id) for cluster_id in clusters]

df['cluster_id'] = clusters
print(f"There are {len(set(clusters))} clusters")

/usr/local/lib/python3.12/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


There are 8 clusters


# Main topics per cluster

In [5]:
# Using the proper c-TF-IDF formular

other_words_to_ignore = ['2025', 'com', 'us', '000', 'ca', 'ch', '10', '41', '91',
                         'bj', 'kakanakou', 'miguel', 'stephane', '94085', '06',
                         'maude', 'asin', '1zwnj000', 'sarl', '12', '30', 'via',
                         'one', 'www', 'fcfa', '38', 'firebasemiguel', 'two',
                         'first', 'd68d8', 'rtdb', '94043', '1600', 'pkwy', '44',
                         '1855', 'rtdblearn', 'amphitheatre', 'pleasecontact',
                         'viewersviewers', 'rise1', '100', 'regardingyour',
                         'projectmanage', 'rulesmigueldatabasemiguel', 'ruleswe',
                         'ürich', 'skakanakou', 'hasinsecure', 'recruiter1', 'ag',
                         'databasecan', 'databasewithout', 'ci', '50', 'solutions1',
                         'sunnyvale', 'west', 'trademarks', 'avenue', 'mosh']

stop_words_arr = []
stop_words_arr.extend(stopwords.words('english'))
stop_words_arr.extend(stopwords.words('french'))
stop_words_arr.extend(stopwords.words('german'))
stop_words_arr.extend(other_words_to_ignore)


# Cluster count
# Substracting 1 because there is a cluster with ID -1
cluster_count = len(set(clusters)) - 1

# Cluster content
cluster_contents = []
for cluster_id in range(cluster_count):
  current_cluster_content_arr = df[df['cluster_id'] == cluster_id]['body'].tolist()
  current_cluster_content = ' '.join(current_cluster_content_arr)
  cluster_contents.append(current_cluster_content)


# Bag of words
count_vectorizer = CountVectorizer(stop_words=stop_words_arr, max_features=10_000)
count_result = count_vectorizer.fit_transform(cluster_contents).toarray()
vocabulary = count_vectorizer.get_feature_names_out()

# Compute the average number of words per class
average_words_per_cluster = sum(sum(count_result)) / len(cluster_contents)

# Compute c-tf-idf
# Final expected shape is [number_of_cluster, vocabulary_size]
# If the word w is not in the cluster, we set its c-TF-IDF value to -1
c_tf_idf_result = []
for cluster_id in range(cluster_count):
  current_cluster_c_tf_idf = []
  for word_idx in range(vocabulary.shape[0]):
    c_tf = count_result[cluster_id][word_idx]
    if c_tf == 0:
      current_cluster_c_tf_idf.append(-1.0)
    else:
      c_tf_idf = average_words_per_cluster / sum(count_result[:, word_idx])
      c_tf_idf = math.log(1 + c_tf_idf)
      c_tf_idf = c_tf * c_tf_idf
      current_cluster_c_tf_idf.append(c_tf_idf)
  c_tf_idf_result.append(current_cluster_c_tf_idf)
c_tf_idf_result = np.array(c_tf_idf_result)


# Get the first top 20 topics per cluster
top_n = 20
k_th = -1 * top_n
top_words_indexes = np.argsort(c_tf_idf_result, axis=1)[:, k_th:][:, ::-1]
top_words = np.take(vocabulary, top_words_indexes)
print(top_words)


[['schule' 'escola' 'einstellungen' 'nachricht' 'planzer' 'falls'
  'adliswil' 'interactive' 'paket' 'brokers' 'sendung' 'template' 'app'
  'folgenden' 'email' 'mitgeliefert' 'angezeigter' 'mailweiterleitung'
  'beantworten' 'benachrichtigungen']
 ['linkedin' 'email' 'engineer' 'corporation' 'logo' 'intended'
  'included' 'unsubscribe' 'notification' 'registered' 'receiving'
  'emails' 'java' 'group' 'software' 'learn' 'medium' 'searches' 'missed'
  'found']
 ['amazon' 'coop' 'goods' 'cancellation' 'service' 'mail' 'contract'
  'please' 'newsletter' 'delivery' 'order' 'inc' 'chf' 'europe'
  'luxembourg' 'eur' 'right' 'plus' 'adresse' 'kennedy']
 ['nvidia' 'courses' 'new' 'learn' 'css' 'learning' 'frontend' 'masters'
  'build' 'skills' 'course' 'certification' 'get' 'year' 'email'
  'updates' 'hands' 'time' 'code' 'javascript']
 ['linkedin' 'new' 'business' 'work' 'help' 'people' 'image' 'get'
  'microsoft' 'like' 'google' 'time' 'openai' 'tools' 'next' 'year'
  'world' 'way' 'career' '

# Generate Labels from the topics

## List of available models

In [7]:
for model in client.models.list():
  print(model.name)

models/embedding-gecko-001
models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-exp-1206
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image-preview
models/gemini-2.5-flash-image
models/gemini-2.5-flash-preview-09-2025
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-flash-preview
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1.5-preview
models/g

In [6]:
MODEL_ID = "gemini-2.5-flash-lite"
# MODEL_ID = "gemini-3-flash-preview"
# MODEL_ID = "gemma-3-27b-it"
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
client = genai.Client(api_key=GEMINI_API_KEY)

prompt_template = """I will provide a list of keywords. Your task is to
generate a single, concise label (1–3 words) that accurately represents the
overarching theme or category of these words. The label should be easy to understand.

Keywords: {}

Think before answering but Output ONLY the label. Do not include introductory
text, quotes, or explanations."""
email_labels = []
total_tokens_arr = []

for cluster_id in range(cluster_count):
  prompt = prompt_template.format(', '.join(top_words[cluster_id].tolist()))
  response = client.models.generate_content(
      model=MODEL_ID,
      contents=prompt)
  email_labels.append(response.text)
  total_tokens_arr.append(response.usage_metadata.total_token_count)

print(email_labels)
print(total_tokens_arr)

['Messaging & School Apps', 'Digital Communication', 'Online Shopping', 'Frontend Skills', 'Future Work', 'Data Systems Engineering', 'AI Models']
[136, 117, 120, 117, 118, 118, 119]


In [12]:
no_cluster_id = -1
emails_clusters_mapping = []
for cluster_id in clusters:
  if cluster_id == no_cluster_id:
    emails_clusters_mapping.append('NO CLUSTER')
  else:
    emails_clusters_mapping.append(email_labels[cluster_id])

df['cluster_name'] = emails_clusters_mapping